# From Prompting to LLM Application 📝
The practical goal of this notebook is to build a Sales Brochure Generator.

Given a company’s website URL, the system will:
1. Retrieve the main webpage.
2. Identify relevant internal links (e.g., About, Products, Mission).
3. Follow those links.
4. Aggregate the collected information.
5. Generate a structured sales brochure targeted at:
        - potential clients,
        - investors,
        - or future team members.

This goes beyond simple summarization.

> We are going to perform multi-page information retrieval + synthesis.

In [1]:
# imports
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


# 🔎 Extracting Links Is Easy — Understanding Them Is Hard

In [4]:
links = fetch_website_links("https://www.bendingspoons.com")
links

['https://bendingspoons.com/',
 '/',
 '/values',
 '/products',
 '/technologies',
 '/people',
 '/events',
 '/careers',
 'https://jobs.bendingspoons.com/?utm_source=bendingspoons&utm_medium=website&utm_content=nav_link',
 'https://bendingspoons.com/',
 '/',
 '/',
 '/values',
 '/products',
 '/technologies',
 '/people',
 '/events',
 '/careers',
 'https://jobs.bendingspoons.com/?utm_source=bendingspoons&utm_medium=website&utm_content=nav_link',
 '/products',
 '/technologies',
 '/people',
 'https://scholarship.bendingspoons.com/',
 'https://employability.bendingspoons.com/',
 '/careers',
 '/',
 '/values',
 '/products',
 '/technologies',
 '/people',
 '/events',
 '/careers',
 'https://jobs.bendingspoons.com/?utm_source=bendingspoons&utm_medium=website&utm_content=footer_link',
 'https://support.bendingspoons.com/compliance.html',
 'https://instagram.com/bendingspoons',
 'https://linkedin.com/company/bendingspoons',
 'https://medium.com/bendingspoons',
 'https://www.glassdoor.it/Reviews/Bending

With the prevous line of code, we retrieve all the links found on a given webpage. However, this raw extraction presents two main challenges:
1. Not all links are relevant: a webpage may contain links to external platforms that are completely unrelated to the wanted information -> Determining relevance requires contextual understanding.
2. Relative vs. absolute URLs: some links are complete (absolute URLs), while others are relative paths that must be reconstructed into full URLs -> Handling this correctly requires additional parsing logic.

## Solution ✅
We can delegate the links interpretation to an LLM. With a small model call, we can filter links based on contextual relevance in a way that resembles human judgment.

> LLMs are not just generating text — they are parsing, interpreting, and restructuring information with semantic awareness.